In [1]:
import pandas as pd
import numpy as np
import id_nodes
import id_changes
import update_bom
import importlib
import format_xlsx

In [2]:
master_bom_path = r"..\ShortMaster-0015442-FS-8100-BOM-20200326.xlsx"
new_bom_path = r"..\ShortTest-0015442-FS-8100-BOM-20200326.xlsx"

In [3]:
master_bom_df = pd.read_excel(master_bom_path)
new_bom_df = pd.read_excel(new_bom_path)

In [4]:
master_bom_df.index.set_names('Order', inplace=True)
new_bom_df.index.set_names('Order', inplace=True)

In [5]:
master_bom_df['Node ID'] = id_nodes.id_nodes(master_bom_df)
new_bom_df['Node ID'] = id_nodes.id_nodes(new_bom_df)

In [6]:
master_bom_df.set_index('Node ID', append=True, inplace=True)
new_bom_df.set_index('Node ID', append=True, inplace=True)

In [7]:
main_widths = [5, 60, 60, 60, 15, 60, 10, 10, 15]
key_width = 23

In [8]:
deleted_nodes = id_changes.get_deleted_nodes(master_bom_df, new_bom_df)
added_nodes = id_changes.get_added_nodes(master_bom_df, new_bom_df)
reordered_nodes = id_changes.get_reordered_nodes(master_bom_df, new_bom_df)

In [9]:
added_columns = id_changes.get_added_columns(master_bom_df, new_bom_df)

In [10]:
updated_elements = id_changes.get_updated_elements(master_bom_df, new_bom_df, order_only=None)

In [11]:
updated_nodes = id_changes.get_updated_nodes(master_bom_df, new_bom_df, order_only=None)

In [12]:
updated_bom_df = update_bom.merge_support_columns(master_bom_df, new_bom_df).reset_index().set_index('Node ID', append=True)

In [13]:
updated_bom_df.index.set_names(['Order', 'Node ID'], inplace=True)

In [14]:
writer = pd.ExcelWriter('pandasEx.xlsx', engine ='xlsxwriter')

In [15]:
updated_bom_df.to_excel(writer, sheet_name ='Updated BOM', index=False)
master_bom_df.loc[(slice(None), deleted_nodes), :].to_excel(writer, sheet_name ='Removed Nodes', index=False)
new_bom_df.loc[(slice(None), added_nodes), :].to_excel(writer, sheet_name ='Added Nodes', index=False)
master_bom_df.loc[(slice(None), updated_nodes), :].to_excel(writer, sheet_name ='Updated Nodes', index=False)
master_bom_df.loc[(slice(None), reordered_nodes), :].to_excel(writer, sheet_name ='Reordered Nodes', index=False)

In [16]:
workbook = writer.book

In [17]:
worksheet_names = [worksheet.name for worksheet in workbook.worksheets()]

In [18]:
added_column_header_format = workbook.add_format({'bg_color': '#ADD8E6', 'bold': True, 'border': 1, 'align': 'center_across', 'valign': 'top'})
added_node_format = workbook.add_format({'bg_color': '#00B0F0'})
reordered_node_format = workbook.add_format({'bg_color': '#92D050'})
updated_element_format = workbook.add_format({'bg_color': '#FFFF00'})

In [19]:
updated_bom_sheet = workbook.get_worksheet_by_name('Updated BOM')

In [20]:
column_map = {column_name: column_position for column_position, column_name in enumerate(updated_bom_df.columns)}

In [21]:
for column_name in added_columns:
    updated_bom_sheet.write(0, updated_bom_df.columns.get_loc(column_name), column_name, added_column_header_format)

In [22]:
for node_position, node_name in enumerate(updated_bom_df.index.get_level_values('Node ID')):
    if node_name in added_nodes:
        updated_bom_sheet.write(node_position + 1, updated_bom_df.columns.get_loc('Level'), updated_bom_df.loc[(slice(None), node_name), 'Level'][0], added_node_format)
        updated_bom_sheet.write_string(node_position + 1, updated_bom_df.columns.get_loc('File Name'), updated_bom_df.loc[(slice(None), node_name), 'File Name'][0], added_node_format)

    if node_name in reordered_nodes:
        updated_bom_sheet.write(node_position + 1, updated_bom_df.columns.get_loc('Level'), updated_bom_df.loc[(slice(None), node_name), 'Level'][0], reordered_node_format)
        updated_bom_sheet.write_string(node_position + 1, updated_bom_df.columns.get_loc('File Name'), updated_bom_df.loc[(slice(None), node_name), 'File Name'][0], reordered_node_format)

    if node_name in updated_nodes:
        for updated_column in updated_elements.loc[node_name, updated_elements.loc[node_name]].index.values:
            updated_bom_sheet.write(node_position + 1, updated_bom_df.columns.get_loc(updated_column), updated_bom_df.loc[(slice(None), node_name), updated_column][0], updated_element_format)

In [23]:
workbook.add_worksheet('Key')

In [24]:
key_sheet = workbook.get_worksheet_by_name('Key')
key_sheet.write(0, 0, 'Re-ordered nodes', reordered_node_format)
key_sheet.write(1, 0, 'Added nodes or columns', added_node_format)
key_sheet.write(2, 0, 'Updated nodes', updated_element_format)
key_sheet.set_column(0, 0, key_width)

0

In [25]:
max_column_widths = format_xlsx.get_max_column_widths(updated_bom_df, buffer=2)

In [26]:
for worksheet in workbook.worksheets():
    if not worksheet.name == 'Key':
        format_xlsx.set_sheet_column_widths(worksheet, max_column_widths)

In [27]:
writer.save()